In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.callbacks import ModelCheckpoint  # 모니터링 모듈
from keras.callbacks import EarlyStopping

import pandas as pd
import numpy as np
import os
import tensorflow as tf

In [ ]:
import platform

path = "c:/Windows/Fonts/malgun.ttf"
from matplotlib import font_manager, rc
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry')

In [ ]:
seed=0
np.random.seed(seed)
tf.set_random_seed(seed)

In [ ]:
df = pd.read_csv('air.csv',encoding='euc_kr',header=0)
air_J = df.sample(frac=1)

air_J = pd.DataFrame(air_J)



air_J = air_J[air_J['결항여부'] != 1]  # 결항 데이터들 삭제
air_J = air_J[air_J['부정기편'] != 1]  # 부정기편 데이터들 삭제

air_J.head()

In [ ]:
air_J.shape

In [ ]:


# 변수 선택하기

air_J = air_J[["지연여부","공항","항공사","편명","출도착","시간대","day_airline","VIS","평균풍속(kts)"]]


air_J = air_J.sort_values(["날짜"], ascending=[True])



In [ ]:
# 날짜를 index로 만들기 - 시계열 데이터
air_J = air_J.set_index('날짜') 

In [ ]:
# 더미 변수
air_J = pd.get_dummies(air_J, columns=['공항', '항공사','편명'])
air_J.head()

In [ ]:
air_J.info()

In [ ]:
x_data = air_J.values[:, [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]]  
y_data = air_J.values[:, [0]]            # 클래스 : 지연여부

In [ ]:
# 훈련집합, 평가집합 나누기

In [ ]:

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, 
                                                    test_size=0.2, random_state=7)

In [ ]:
# 모델 설정

model = Sequential()
model.add(Dense(12, input_dim=29, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1,  activation='sigmoid'))

In [ ]:
# 모델 컴파일

model.compile(loss='mean_squared_error', 
              optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
# 모델 실행 및 저장

model.fit(x_train, y_train, 
          epochs=200, 
          batch_size=10)  # 10개의 행데이터를 단위로 200번 수행함.  
          

In [ ]:
scores = model.evaluate(x_test, y_test)
print(scores)

In [ ]:
# predict 메서드를 호출하여 test_x의 값으로 모델을 테스트한 다음, 
# score 메서드를 사용하여 모델의 평균 정확도를 확인합니다.
predicted = model.predict(x_test)
#model.score(x_test, y_test)

In [ ]:
# 평가집합에서 예측 확률 세트를 생성
from sklearn.metrics import roc_auc_score
probabilities = model.predict_proba(x_test)

In [ ]:
# roc auc 점수 생성 
# 데이터의 불균형으로 인해 항공편의 지연 도착 예측보다 항공편의 정시 도착 예측이 정확할 가능성이 더 높습니다.
# ROC AUC는 이를 고려하며, 정시 또는 지연 도착에 대한 예측이 정확할 가능성을 더 정확하게 나타냅니다.
roc_auc_score(y_test, probabilities[:, 0])

In [ ]:
# 혼동 행렬 생성 - 혼동 행렬은 각 답변이 정확하거나 잘못 분류된 횟수를 정량화합니다. 
# 특히 거짓 긍정, 거짓 부정, 참 긍정 및 참 부정의 수를 정량화합니다. 
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predicted)

In [ ]:
# 정밀도 계산 - precision score
from sklearn.metrics import precision_score

train_predictions = model.predict(x_train)
precision_score(y_train, train_predictions)

In [ ]:
# 회수 계산용 -recall score
from sklearn.metrics import recall_score

recall_score(y_train, train_predictions)

In [ ]:
####  실험 데이터 예측 해보기...

test = np.array([# 속성 값들 입력해라]).reshape(1,10)
model.predict(test)